In [1]:
from functools import partial

import numpy as np
import seaborn as sns
from ipywidgets import FloatSlider, IntSlider, interact, interactive, Checkbox, Textarea, Button, FloatText, VBox, HBox
from matplotlib import pyplot as plt

import filters
import noise
from utils import imshow, panels, convolution
from filters import harris, DoG, LoG, gesse

In [2]:
WindowSlider = partial(
    IntSlider, value=3, min=1, max=21, step=2, description="Окно"
)
RadiusSlider = partial(FloatSlider, min=0.1, value=1, max=5)
AmountSlider = partial(FloatSlider, value=1, min=0, max=5)
BoolSlider = partial(Checkbox, value=False)

In [3]:
sns.set_theme()

In [4]:
image = plt.imread('a.png')[:,:,:3]
# image = plt.imread('lenna.jpg') / 256

## 1.


In [5]:
@interact
def inner(window=WindowSlider(), sigma=RadiusSlider(), amount=AmountSlider(),toggle=BoolSlider()):
    left, right = panels()
    imshow(filtred := filters.average(image, window) if toggle else filters.blur(image, sigma), left)
    imshow(np.abs(image-filtred*amount), right)

interactive(children=(IntSlider(value=3, description='Окно', max=21, min=1, step=2), FloatSlider(value=1.0, de…

## 2.


In [6]:
def get_matrix_array(matrix_widgets):
    matrix = np.array([[widget.value for widget in row.children] for row in matrix_widgets.children])
    return matrix

def create_matrix_input(n):
    matrix = []
    
    for i in range(n):
        row = []
        for j in range(n):
            row.append(FloatText(value=1))#, description=f'({i},{j}):'))
        matrix.append(row)
    matrix_layout = VBox([HBox(row) for row in matrix])
    return matrix_layout

In [7]:
def second(normalize,half,kernel):
    img = image.copy()
    img = convolution(img,kernel)
    if half:
        img += 0.5
    if normalize:
        mx = np.max(img)
        img = img * 1 / mx if mx > 0 else img  
    imshow(img)
    

In [8]:
@interact
def inner(
    normalize=Checkbox(
        value=True,
        description="Нормализовать?"
        ),
    half=Checkbox(
        value=False,
        description="Добавить половину"
    ),
    n=IntSlider(
        value=3,
        min=1,
        max=20,
        description="N="
    )
    
):
    matrix_widgets = create_matrix_input(n)  
    display(matrix_widgets)    
    button = Button(description="Пересчитать")
    display(button)
    
    kernel = np.array([[float(widget.value) for widget in row.children] for row in matrix_widgets.children])
    
    
    second(normalize,half,kernel)
    def on_button_click(b):
        values = [[float(widget.value) for widget in row.children] for row in matrix_widgets.children]
        kernel = np.array(values)
        display(kernel)
        second(normalize,half,kernel)
        
    button.on_click(on_button_click)


interactive(children=(Checkbox(value=True, description='Нормализовать?'), Checkbox(value=False, description='Д…

# 3. Харрис

In [9]:
@interact
def inner(
    treshhold=FloatSlider(
        value=0.01,
        min=0.01,
        max=0.2,
        step=0.01
    ),
    a=FloatSlider(
        value=0.04,
        min=0.04,
        max=0.06,
        step=0.01
    )
):
    imshow(harris(image,threshold=treshhold,a=a))

interactive(children=(FloatSlider(value=0.01, description='treshhold', max=0.2, min=0.01, step=0.01), FloatSli…

### Гессе

In [10]:
@interact
def inner(
    treshhold=FloatSlider(
        value=0.01,
        min=0.01,
        max=1,
        step=0.01
    )
):
    imshow(gesse(image,threshold=treshhold))

interactive(children=(FloatSlider(value=0.01, description='treshhold', max=1.0, min=0.01, step=0.01), Output()…

# 4. Dog

In [ ]:
@interact
def inner(
    sigma=FloatSlider(
        value=0.3,
        min=0.01,
        max=3,
        step=0.01
    ),
    a=FloatSlider(
        value=1.6,
        min=0.1,
        max=10,
        step=0.1
    )
):
    imshow(DoG(image,sigma=sigma,alpha=a))

interactive(children=(FloatSlider(value=0.3, description='sigma', max=3.0, min=0.01, step=0.01), FloatSlider(v…

### LoG

In [12]:
@interact
def inner(
    sigma=FloatSlider(
        value=0.3,
        min=0.01,
        max=3,
        step=0.01
    ),
    a=FloatSlider(
        value=1.6,
        min=0.1,
        max=10,
        step=0.1
    )
):
    imshow(LoG(image,sigma=sigma,alpha=a))

interactive(children=(FloatSlider(value=0.3, description='sigma', max=3.0, min=0.01, step=0.01), FloatSlider(v…